In [ ]:
import os 
import rootutils
import open3d as o3d
import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict

rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.utils import remove_small_clusters

In [ ]:
target_pcd = o3d.io.read_point_cloud('../output/pcd/20250429_105703_0.ply')
target_pcd = remove_small_clusters(target_pcd, voxel_size=0.01, min_cluster_size=200)
o3d.visualization.draw_geometries([target_pcd])

In [ ]:
def split_point_cloud(pcd: o3d.geometry.PointCloud, num_splits: tuple[int, int, int]) -> list[o3d.geometry.PointCloud]:
    """
    Split a point cloud into `num_splits` equal parts along the x-axis.
    
    Args:
        pcd (o3d.geometry.PointCloud): The input point cloud.
        num_splits tuple(int, int, int): The number of splits to create.
    
    Returns:
        list: A list of point clouds, each representing a split part.
    """
    min_bound = pcd.get_min_bound()
    max_bound = pcd.get_max_bound()

    x_step = (max_bound[0] - min_bound[0]) / num_splits[0]
    y_step = (max_bound[1] - min_bound[1]) / num_splits[1]
    z_step = (max_bound[2] - min_bound[2]) / num_splits[2]
    
    split_pcds = []
    
    for i in range(num_splits[0]):
        for j in range(num_splits[1]):
            for k in range(num_splits[2]):
                # Create bounding box for each sub-region
                bbox_min = min_bound + [i * x_step, j * y_step, k * z_step]
                bbox_max = min_bound + [(i + 1) * x_step, (j + 1) * y_step, (k + 1) * z_step]
                aabb = o3d.geometry.AxisAlignedBoundingBox(bbox_min, bbox_max)

                # Crop the point cloud
                cropped = target_pcd.crop(aabb)
                if len(cropped.points) > 0:
                    split_pcds.append(cropped)
    
    return split_pcds


def split_point_cloud_by_kmeans(pcd: o3d.geometry.PointCloud, num_clusters: int) -> list[o3d.geometry.PointCloud]:
    """
    Split a point cloud into `num_clusters` using KMeans clustering on XYZ coordinates.
    
    Args:
        pcd (o3d.geometry.PointCloud): The input point cloud.
        num_clusters (int): Number of clusters to create.
    
    Returns:
        list[o3d.geometry.PointCloud]: A list of clustered point clouds.
    """
    points = np.asarray(pcd.points)
    
    if len(points) < num_clusters:
        raise ValueError("Number of clusters exceeds number of points.")

    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(points)

    split_pcds = []
    for i in range(num_clusters):
        indices = np.where(labels == i)[0]
        cluster = pcd.select_by_index(indices)
        split_pcds.append(cluster)
    
    return split_pcds


def split_point_cloud_by_voxel_grid(pcd: o3d.geometry.PointCloud, voxel_size: float) -> list[o3d.geometry.PointCloud]:
    """
    Split a point cloud into voxel-based groups without downsampling.
    
    Args:
        pcd (o3d.geometry.PointCloud): The input point cloud.
        voxel_size (float): Size of the voxel to define group resolution.
    
    Returns:
        list[o3d.geometry.PointCloud]: A list of point clouds, each corresponding to one voxel group.
    """
    points = np.asarray(pcd.points)
    
    # Compute voxel indices for each point
    voxel_indices = np.floor(points / voxel_size).astype(np.int32)
    voxel_keys = [tuple(v) for v in voxel_indices]

    voxel_dict = defaultdict(list)
    for idx, key in enumerate(voxel_keys):
        voxel_dict[key].append(idx)

    # Create point clouds for each voxel group
    split_pcds = []
    for indices in voxel_dict.values():
        sub_pcd = pcd.select_by_index(indices)
        split_pcds.append(sub_pcd)

    return split_pcds

In [ ]:
split_pcds = split_point_cloud(target_pcd, num_splits=(3, 3, 3))
# Assign each split a random color
for pcd in split_pcds:
    color = np.random.rand(3)
    pcd.paint_uniform_color(color)
# visualize the split point clouds
o3d.visualization.draw_geometries(split_pcds)

In [ ]:
split_pcds = split_point_cloud_by_kmeans(target_pcd, num_clusters=8)
# Assign each split a random color
for pcd in split_pcds:
    color = np.random.rand(3)
    pcd.paint_uniform_color(color)
# visualize the split point clouds
o3d.visualization.draw_geometries(split_pcds)

In [ ]:
split_pcds = split_point_cloud_by_voxel_grid(target_pcd, voxel_size=0.5)
# Assign each split a random color
for pcd in split_pcds:
    color = np.random.rand(3)
    pcd.paint_uniform_color(color)
# visualize the split point clouds
o3d.visualization.draw_geometries(split_pcds)